# Machine Learning on Perovskites Stability

## Data Import

In [ ]:
## import data
import numpy as np
import pandas as pd
%matplotlib inline
data =  pd.read_csv('new_stability_output_with_descriptors.csv', index_col=0) 
cdata = pd.read_csv('continuous_stability_output_with_descriptors.csv',index_col=0)
ddata = pd.read_csv('disc_stability_output_with_descriptors.csv',index_col=0)
X = data[data.columns[0:-1]]
Xc = cdata[cdata.columns[0:-1]]
Xd = ddata[ddata.columns[0:-1]]
y = cdata[cdata.columns[-1]]
## store yl as continuous target for regression work
mdata = pd.read_csv('continuous_stability_output_with_descriptors.csv',index_col=0)
yl = mdata[mdata.columns[-1]]
# label the instance for classification work
# Energy above 40 meV is considered to be unstable
y[y<=40] = 1
y[y>40] = 0

In [ ]:
import matplotlib.pyplot as plt
yhist = yl.plot.hist(bins=1000)
plt.xlim(0,400)
plt.xlabel('Energy above hull')

## Data Preprocessing

In [ ]:
from sklearn import preprocessing
## scale the features
## This part is a little cheating, need to be visited later. But currently, this won't affect much.
Xc_scaled = preprocessing.scale(Xc)
Xd_scaled = preprocessing.scale(Xd)
## remove constant feature
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold()
Xc_ns = sel.fit_transform(Xc_scaled)
Xd_ns = sel.fit_transform(Xd_scaled)
## do PCA learning, keep 0.99 variance
## select K-best, keep top 10 score features
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion
import pylab as pl
pca = PCA(n_components=0.99)
selection = SelectKBest(k=3)
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
Xc_con = combined_features.fit_transform(Xc_ns, y)
Xd_con = combined_features.fit_transform(Xd_ns, y)
X_features = np.concatenate((Xc_con,Xd_con),axis=1)

## Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

crand = RandomForestClassifier(n_estimators=100)
mycv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
scores = cross_val_score(crand, X_features, y, cv = mycv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.neural_network import MLPClassifier
cmlp = MLPClassifier(activation='relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(35,8), max_iter=700)
mycv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
scores = cross_val_score(cmlp, X_features, y, cv = mycv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Result Analysis

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X_features, y, test_size=0.2, random_state=300, stratify = y)

cmlp.fit(X_train,y_train)
y_true, y_pred = y_train, cmlp.predict(X_train)
print(confusion_matrix(y_true, y_pred))
y_true, y_pred = y_test, cmlp.predict(X_test)
print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X_features, y, test_size=0.2, random_state=0, stratify = y)

# Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [10,30,50, 70,100,150],
                    'class_weight' : ['balanced', None]}

scores = ['precision', 'accuracy']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RandomForestClassifier(n_jobs=4), tuned_parameters, cv=mycv,
                       scoring=score, n_jobs = 4)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
## ROC CUrve
from sklearn.metrics import roc_curve,auc
clf = RandomForestClassifier(n_estimators=70,class_weight='balanced',n_jobs=4)
y_score = clf.fit(X_train, y_train).predict_proba(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    ytrue = np.array(y_test==i,dtype=int).reshape(-1,1)
    fpr[i], tpr[i], _ = roc_curve(ytrue, y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve stable 1' )
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve stable 0' )
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic RF')
plt.legend(loc="best")
plt.show()

In [ ]:
## validation Curve
from sklearn.model_selection import validation_curve
from sklearn.utils import shuffle
X_shuffled, y_shuffled = shuffle(X_features, y, random_state=0)
my_param_range=[20,30,50,70,90,120,150,200,300]
train_scores, valid_scores = validation_curve(clf, X_shuffled, y_shuffled, 
                                              param_name="n_estimators", 
                                              param_range=my_param_range, cv= mycv, scoring="precision",
                                              n_jobs = 4)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)

import matplotlib.pyplot as plt

plt.title("Validation Curve with RF")
plt.xlabel("number of estimators")
plt.ylabel("Precision Score")
plt.ylim(0.0, 1.1)
lw = 2
param_range=my_param_range
plt.plot(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.plot(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

title = "Learning Curves (Random Forests)"
plot_learning_curve(clf, title, X_shuffled, y_shuffled, ylim=(0.5, 1.01), cv=mycv, n_jobs=4)

In [ ]:
import pylab as pl
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
clf = RandomForestClassifier(n_estimators=100)
## clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-3, n_iter=5, random_state=34)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=clf, step=1, cv=mycv,
              scoring='accuracy')
rfecv.fit(X_features, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
pl.xlabel("Number of features selected")
pl.ylabel("Cross validation score (nb of correct classifications)")
pl.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
pl.show()

In [ ]:
pl.xlabel("Number of features selected")
pl.ylabel("Cross validation score (nb of correct classifications)")
pl.plot(range(1, 20 + 1), rfecv.grid_scores_[0:20])
pl.show()

## Regression

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(activation='relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(35,8), max_iter=700)
clf.fit(X_train, y_train)
y_plin = clf.predict(X_test)
pl.plot (y_test, y_plin, "bo", label='lin model')
pl.ylim((-40,700))
pl.xlim((-5, 700))
pl.xlabel("EaboveHull")
pl.ylabel("Predicted")

In [ ]:
y_plin = clf.predict(X_train)
pl.plot (y_train, y_plin, "bo", label='lin model')
pl.ylim((-40,700))
pl.xlim((-5, 700))
pl.xlabel("EaboveHull")
pl.ylabel("Predicted")